In [13]:
import re
import difflib


In [14]:
def extract_functions(file_content):
    # 正则表达式模式以匹配C/C++函数声明
    function_pattern = r'(\w+)\s+(\w+)\s*\(([^)]*)\)\s*{'
    functions = re.findall(function_pattern, file_content)
    return functions

In [15]:
def find_function_body(return_type, function_name, functions_lines):
    key = return_type + " " + function_name
    start_line = [index for index, item in enumerate(functions_lines, 0) if key in item][0]
    in_function_body = False
    brace_count = 0  # 用于跟踪花括号的嵌套层级
    for i, line in enumerate(functions_lines[start_line:]):
        if not in_function_body:
            if '{' in line:
                in_function_body = True
                brace_count = 1
        else:
            if '{' in line:
                brace_count = brace_count + 1
            if "}" in line:
                brace_count = brace_count - 1
            if brace_count == 0:
                if start_line + i < len(functions_lines):
                    return functions_lines[start_line:start_line + i + 1]
                else:
                    return functions_lines[start_line:-1]

In [16]:
def find_called_functions(function_body):
    # 正则表达式模式以匹配函数调用
    call_pattern = r'\b(\w+)\s*\('
    called_functions = re.findall(call_pattern, function_body)
    return called_functions


In [17]:
with open('foo.c', 'r') as file:
    file_content = file.read()

cleaned_code = re.sub(r'/\*.*?\*/', '', file_content, flags=re.DOTALL)  # 使用正则表达式匹配和删除多行注释
cleaned_code = re.sub(r'//.*', '', cleaned_code)  # 使用正则表达式匹配和删除单行注释

functions_lines = [item for item in cleaned_code.split('\n') if item]
functions = extract_functions(file_content)
function_dic = {}

FileNotFoundError: [Errno 2] No such file or directory: 'foo.c'

In [ ]:
#accessing the return type
func_num = 0
first_function_return_type = functions[0][0]
#checking the file type that needs to be created
file_extension = 'cpp' if 'cpp' in file.name else 'c'
output_file_name = f"equivalencetest.{file_extension}"

#writing the main
with open(output_file_name, 'w') as output_file:
    output_file.write("\n")
    output_file.write("\nint main()\n{\n")
    output_file.write("    " + str(first_function_return_type) + " x;\n")
    output_file.write("    s2e_make_symbolic(&x, sizeof(x), \"x\");\n")

    func_calls = []
    for func_num, (return_type, function_name, function_para) in enumerate(functions):
        call_str = f"    {return_type} func{func_num} = {function_name}(x);\n"
        output_file.write(call_str)
        func_calls.append(f"func{func_num}")


    if len(func_calls) >= 2:
        # Create a comparison string that compares all function calls for equality
        comparison_string = " == ".join(func_calls)
        assert_statement = f"    s2e_assert({comparison_string});\n"
        output_file.write(assert_statement)

    output_file.write("    return 0;\n")
    output_file.write("}\n")



In [ ]:
def calculate_diff(function1, function2):
    # 使用difflib计算两个函数之间的差异
    f1=str(function_dic[function1])
    f2=str(function_dic[function2])
    differ = difflib.Differ()
    diff = differ.compare(f1, f2)
    return '\n'.join(diff)


keys_list = list(function_dic.keys())
differ=calculate_diff(keys_list[0],keys_list[1])
print(differ)


I used Python and regular expressions to complete the task, but I don't think this is the best solution. After writing this code, I also searched for other methods, including using clang and pycparser to generate abstract syntax trees to achieve the same effect. However, I believe that the form of regular expressions is the best way for beginners, especially for query question 4, I think there may be some issues. I think it may be a practical issue with Jupyter Lab.